Test the fama_french module

In [1]:
# import functions from utils and porteval
import sys
import os

from portopt.fama_french import load_ff_3factor_daily
from portopt.fama_french import asset_class_proxy_returns
from portopt.fama_french import asset_class_proxy_prices
from portopt.fama_french import multi_asset_class_proxy_returns
from portopt.fama_french import multi_asset_class_proxy_prices
from portopt.market_data import get_tickers_data
from portopt.market_data import get_portfolio_data
from portopt.utils import standardize_data
from portopt.utils import plot_time_series
from portopt.porteval import simulate_model_portfolio

In [ ]:
# load the Fama-French 3 Factor data
ff_3factor_data = load_ff_3factor_daily()
plot_time_series(ff_3factor_data)


In [ ]:
# generate asset class proxy returns
proxy_returns = asset_class_proxy_returns(ff_3factor_data, 1.0, 1.2, 0.8)
plot_time_series(proxy_returns)

In [ ]:
proxy_prices = asset_class_proxy_prices(ff_3factor_data, 1.0, 1.2, 0.8)
plot_time_series(proxy_prices)

In [ ]:
# multiple asset class proxy returns
coeff_dict = {
    "Large Cap Value": {"beta_m": 1.0, "beta_s": -0.2, "beta_v": 0.5},
    "Large Cap Core": {"beta_m": 1.0, "beta_s": 0.0, "beta_v": 0.0},
    "Large Cap Growth": {"beta_m": 1.0, "beta_s": -0.2, "beta_v": -0.5},
    "Mid Cap Value": {"beta_m": 1.1, "beta_s": 0.4, "beta_v": 0.5},
    "Mid Cap Core": {"beta_m": 1.05, "beta_s": 0.3, "beta_v": 0.2},
    "Mid Cap Growth": {"beta_m": 1.0, "beta_s": 0.35, "beta_v": -0.3},
    "Small Cap Value": {"beta_m": 1.2, "beta_s": 0.8, "beta_v": 0.7},
    "Small Cap Core": {"beta_m": 1.15, "beta_s": 0.7, "beta_v": 0.2},
    "Small Cap Growth": {"beta_m": 1.1, "beta_s": 0.85, "beta_v": -0.4},
}
multi_asset_proxy_returns = multi_asset_class_proxy_returns(ff_3factor_data, coeff_dict)
plot_time_series(multi_asset_proxy_returns)

In [ ]:
# multiple asset class proxy prices
multi_asset_proxy_prices = multi_asset_class_proxy_prices(ff_3factor_data, coeff_dict)
plot_time_series(multi_asset_proxy_prices)


In [ ]:
moderate_portfolio = {
    "Large Cap Value": 0.10,
    "Large Cap Core": 0.19,
    "Large Cap Growth": 0.13,
    "Mid Cap Value": (0.14/3),
    "Mid Cap Core": (0.14/3),
    "Mid Cap Growth": (0.14/3),
    "Small Cap Value": (0.07/3),
    "Small Cap Core": (0.07/3),
    "Small Cap Growth": (0.07/3),
}
moderate_portfolio

In [ ]:
aggressive_portfolio = {
    "Large Cap Value": 0.11,
    "Large Cap Core": 0.22,
    "Large Cap Growth": 0.13,
    "Mid Cap Value": (0.18/3),
    "Mid Cap Core": (0.18/3),
    "Mid Cap Growth": (0.18/3),
    "Small Cap Value": (0.10/3),
    "Small Cap Core": (0.10/3),
    "Small Cap Growth": (0.10/3),
}
aggressive_portfolio

In [ ]:
vanguard_etfs_moderate = {
    "VTV": 0.10,     # large cap value
    "VV": 0.19,      # large cap core/blend
    "VUG": 0.13,     # large cap growth
    "VOE": (0.14/3), # mid-cap value
    "VO": (0.14/3),  # mid-cap core/blend
    "VOT": (0.14/3), # mid-cap growth
    "VBR": (0.07/3), # small-cap value
    "VB": (0.07/3),  # small-cap core/blend
    "VBK": (0.07/3), # small-cap growth
}
vanguard_etfs_moderate

In [ ]:
vanguard_etfs_aggressive = {
    "VTV": 0.11,     # large cap value
    "VV": 0.22,      # large cap core/blend
    "VUG": 0.13,     # large cap growth
    "VOE": (0.18/3), # mid-cap value
    "VO": (0.18/3),  # mid-cap core/blend
    "VOT": (0.18/3), # mid-cap growth
    "VBR": (0.10/3), # small-cap value
    "VB": (0.10/3),  # small-cap core/blend
    "VBK": (0.10/3), # small-cap growth
}
vanguard_etfs_aggressive

In [ ]:
vanguard_etfs = {
    "Vanguard Moderate": vanguard_etfs_moderate,
    "Vanguard Aggressive": vanguard_etfs_aggressive,
}
vanguard_etfs

In [ ]:
# get prices for vanguard etfs
vanguard_etf_prices, price_data_status = get_portfolio_data(vanguard_etfs, start_date=multi_asset_proxy_prices.index.min(), end_date=None)
vanguard_etf_prices


In [ ]:
# combine vanguard etf prices with proxy prices
combined_prices = vanguard_etf_prices.join(multi_asset_proxy_prices, how="inner")
combined_prices

In [ ]:
#    "VTV"  # large cap value
#    "VV":  # large cap core/blend
#    "VUG": # large cap growth
#    "VOE": # mid-cap value
#    "VO":  # mid-cap core/blend
#    "VOT": # mid-cap growth
#    "VBR": # small-cap value
#    "VB":  # small-cap core/blend
#    "VBK": # small-cap growth

# compare vanguard etf prices with proxy prices
combined_prices_std = standardize_data(combined_prices)
plot_time_series(combined_prices_std[["VTV", "Large Cap Value"]])
plot_time_series(combined_prices_std[["VV", "Large Cap Core"]])
plot_time_series(combined_prices_std[["VUG", "Large Cap Growth"]])
plot_time_series(combined_prices_std[["VOE", "Mid Cap Value"]])
plot_time_series(combined_prices_std[["VO", "Mid Cap Core"]])
plot_time_series(combined_prices_std[["VOT", "Mid Cap Growth"]])
plot_time_series(combined_prices_std[["VBR", "Small Cap Value"]])
plot_time_series(combined_prices_std[["VB", "Small Cap Core"]])
plot_time_series(combined_prices_std[["VBK", "Small Cap Growth"]])


In [ ]:
# Define the pairs of columns to compare
pairs = [
#    ("VTV", "Large Cap Value"),
#    ("VV", "Large Cap Core"),
    ("VUG", "Large Cap Growth"),
#    ("VOE", "Mid Cap Value"),
#    ("VO", "Mid Cap Core"),
#    ("VOT", "Mid Cap Growth"),
#    ("VBR", "Small Cap Value"),
#    ("VB", "Small Cap Core"),
#    ("VBK", "Small Cap Growth")
]

# Create a new DataFrame with the percentage differences
import pandas as pd
differences = pd.DataFrame()
for etf, ff in pairs:
    col_name = f"{etf} vs {ff}"
    differences[col_name] = ((combined_prices_std[ff] - combined_prices_std[etf]) / combined_prices_std[etf]) * 100

# View the results
print("Percentage Difference columns:", differences.columns)
print("\nFirst few rows:")
print(differences.head())

# Plot the differences
plot_time_series(differences, title="% Difference: Fama-French vs ETF Prices (relative to ETF)")

In [ ]:
portfolios ={
    "Moderate": moderate_portfolio,
    "Aggressive": aggressive_portfolio,
    "Vanguard Moderate": vanguard_etfs_moderate,
    "Vanguard Aggressive": vanguard_etfs_aggressive,
}
portfolios


In [ ]:
backtest_results = simulate_model_portfolio(portfolios,
                         start_date=multi_asset_proxy_prices.index.min(),
                         end_date=None,
                         rebalance_freq="quarterly",
                         price_data=combined_prices_std,
                         #engine="bt",
                         verbose=False)
backtest_results.display()
plot_time_series(backtest_results.prices)